In [30]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img



In [31]:
data_folder = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"

In [32]:
def load_images_rgb(folder_path, num_images=3):
    images = []
    labels = []
    
    count = 0  # Counter to load only the specified number of images
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder_path, filename)
            
            # Read and resize the image to match VGG16 input size
            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img)
            
            # Normalize pixel values to be between 0 and 1
            normalized_img = img_array / 255.0
            
            # Append the image and label to the lists
            images.append(normalized_img)
            labels.append(1)  # You can set a label indicating the presence of a face
            
            count += 1
            if count == num_images:
                break
    
    return np.array(images), np.array(labels)

# Load and preprocess three images with RGB channels
images_rgb, labels_rgb = load_images_rgb(data_folder, num_images=10000)


In [33]:
train_images_rgb, val_images_rgb, train_labels_rgb, val_labels_rgb = train_test_split(images_rgb, labels_rgb, test_size=0.2, random_state=42)
print("Training images shape:", train_images_rgb.shape)
print("Validation images shape:", val_images_rgb.shape)


Training images shape: (8000, 224, 224, 3)
Validation images shape: (2000, 224, 224, 3)


In [34]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model using the VGG16 base and add your own layers
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images_rgb, train_labels_rgb, epochs=10, validation_data=(val_images_rgb, val_labels_rgb))

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_images_rgb, val_labels_rgb)
print(f"Validation accuracy: {val_acc}")


Epoch 1/10
250/250 [==============================] - 894s 4s/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
250/250 [==============================] - 1035s 4s/step - loss: 6.8323e-37 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
250/250 [==============================] - 863s 3s/step - loss: 6.8323e-37 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
250/250 [==============================] - 868s 3s/step - loss: 6.8325e-37 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
250/250 [==============================] - 862s 3s/step - loss: 6.8328e-37 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
250/250 [==============================] - 1548s 6s/step - loss: 6.8328e-37 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
250/250 [==============================] - 3817s 15s/step - loss: 6.8323e-37 - accura

In [35]:
model.save("face_detection_model1.h5")

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
model = load_model("face_detection_model1.h5")
face = cv2.imread("/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_img.jpg")
mountains_image = cv2.imread("/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/pexels-pixabay-417173.jpg")
resized_mountains = cv2.resize(mountains_image, (224, 224))
normalized_mountains = resized_mountains / 255.0
input_mountains = normalized_mountains.reshape(1, 224, 224, 3)  # Assuming the model expects 3 channels

prediction_mountains = model.predict(input_mountains)
print("Face detected on mountains image:", prediction_mountains[0][0] >= 1)

1/1 [==============================] - 0s 156ms/step
Face detected on mountains image: True


In [16]:
model = load_model("face_detection_model.h5")

def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

# Load the larger image
large_image = cv2.imread("/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/pexels-pixabay-417173.jpg")

# Define the window size and step size
window_size = (218, 178)
step_size = 10

# Create a flag to indicate if a face is detected
face_detected = False

# Iterate over windows and make predictions
for (x, y, window) in sliding_window(large_image, step_size, window_size):
    # Resize the window to match the input shape expected by the model
    resized_window = cv2.resize(window, (218, 178))
    
    # Preprocess the window
    window_gray = cv2.cvtColor(resized_window, cv2.COLOR_BGR2GRAY)
    window_normalized = window_gray / 255.0
    window_input = window_normalized.reshape(1, 218, 178, 1)

    # Make predictions using the model
    prediction = model.predict(window_input)

    # Assuming 0.5 as the threshold for face detection
    if prediction[0][0] > 0.9:
        face_detected = True
        break  # Exit the loop if a face is detected

# Output True if a face is detected, False otherwise
print("Face detected:", face_detected)

1/1 [==============================] - 0s 39ms/step
Face detected: True
